# Diffie Hellman

In [9]:
from cryptography.hazmat.backends import default_backend
from  cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives.kdf.hkdf import HKDF

# Parametros reusables
# Deben ser iguales en ambos lados
parameters = dh.generate_parameters(generator = 2, key_size = 2048, backend = default_backend())

private_key_emisor = parameters.generate_private_key()
# Esta es la que se tiene que intercambiar
public_key_emisor = private_key_emisor.public_key()


private_key_receptor = parameters.generate_private_key() # los parámetros se pueden reusar
public_key_receptor = private_key_receptor.public_key() # si se va a intercambiar se tiene que serializar primero
# print(public_key_receptor)
# print(private_key_emisor)

shared_key_emisor = private_key_emisor.exchange(public_key_receptor) # esto es binario
shared_key_receptor = private_key_receptor.exchange(public_key_emisor)

print(shared_key_emisor == shared_key_receptor)
print(shared_key_emisor)
print(shared_key_receptor)

ValueError: ('Error computing shared key.', [_OpenSSLErrorWithText(code=101298329, lib=6, reason=153, reason_text=b'error:0609B099:digital envelope routines:EVP_PKEY_derive_set_peer:different parameters')])

## Derivar una llave

- El binario obtenido al crear el secreto compartido es más largo de lo que se necesita
- HKDF es un algoritmo para derivar una llave a través de un secreto (podría ser también una contraseña)

In [2]:
# Hay que convertir el binario compartido a una llave aes apropiada
derived_key = HKDF(algorithm = hashes.SHA256(), 
                length = 32, 
                salt = None,
                info = b'handshake data', # tiene que ser lo mismo de los dos lados
                backend = default_backend()).derive(shared_key_emisor)
                
print(derived_key)
print(len(derived_key)) # apropiada para aes 128
    

b'n\x8a\x7f\xe2K\xcaF\x83\xf1k~\xe3\x10\x99ZK\xab\x0cd\xb3B]\x88\xefa\xfbZ\xa8x\x8ahc'
32


## Serializar llaves

- Necesario para intercambiar la llave pública y poder generar el secreto compartido
- Se muestra como serializar en formato PEM

In [3]:
from cryptography.hazmat.primitives import serialization

llave_serialiada = public_key_emisor.public_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo)
print(llave_serialiada)

# regresar a objeto llave
public_key_recuperada = serialization.load_pem_public_key(
    llave_serialiada,
    backend=default_backend())
print(public_key_recuperada)

b'-----BEGIN PUBLIC KEY-----\nMIICJDCCARcGCSqGSIb3DQEDATCCAQgCggEBANPwFco3LFzR2ZTPeaWjBzEI0JeY\n2MJUbx4q7CKIJV3lQUcOZHw5TOcPctruFYtqIdqcdQ912y9iptKStkc2Ir4zOLUz\nOxC+UKcoIYle9B24BbPLkfAYvmpmNSxWpIzVtxw6RSR6dtJaJIqbsqyKNslHYiWk\nmljrSfXFMAHCbYN9F6CJdKuPn9ZO/6ErpS1ct8iW1eEwHKLKkeID54sRhezsUHZZ\niG2PwXvGANtG0dZBA04rn87Drf3VZbTczDteWuVR8aW8H8qFOPvHfvdKNKsFsOZF\nGpYa4MHkxAGhjtnXSIOkwe5Dp1WRpv8HXDRUNIqX9xfPcxV8YOV+CgGEj0MCAQID\nggEFAAKCAQB0nNvFYar3U5EFnyXIsPZvt8cEmcb4aVOzZXU1JdDlqL46OFF6oZZO\nBVeCqn4ylsPLQxKZIJglS9QC38w+UMFKWRaFXKBHOwQrCCtFiFSi0AhAkYazba1R\nFwdjEBvugITLlu+0CxqyYnqgXMa0DIAQSvu7uuZcV0mfAru0qYtxoDe+IKQdlx0B\nhP49Ixu7ONb1g3GCE9NeNYdcRhqXfn8FgdNgPpt3JpXF8xly4GNAulVTK3xFIFH2\n0FLqmqswFjLjfIluDW5QltMM3+kQGHQj7gdc+qUKxkqLEe5Kd8RvetcWoo+IHkQ3\n+b8YVjoOo9Msy8tegiE5sGhD6Uz560bU\n-----END PUBLIC KEY-----\n'


## ECDH

In [22]:
from cryptography.hazmat.backends import default_backend
from  cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives.kdf.hkdf import HKDF

private_key_emisor = ec.generate_private_key(ec.SECP384R1(), default_backend())
public_key_emisor = private_key_emisor.public_key()

private_key_receptor = ec.generate_private_key(ec.SECP384R1(), default_backend())
public_key_receptor = private_key_receptor.public_key()

shared_key_emisor = private_key_emisor.exchange(ec.ECDH(), public_key_receptor) # esto es binario
shared_key_receptor = private_key_receptor.exchange(ec.ECDH(), public_key_emisor)

print(shared_key_emisor == shared_key_receptor)
print(shared_key_emisor)

True
b"5\x16\xf4~}\x0b'3\xa1]\x14\xc3\x8aF\xd0\xf7\x90s\x97R\xc1\x9cXU\t\xb9!xH3\x96\x01\x88\x15\xa7\x12`D\xa6\xbe\x9b2O\x1ev\n\x8cR"
